In [16]:
import pandas as pd
import numpy as np

import os
from dotenv import load_dotenv
load_dotenv()

from spotipy.oauth2 import SpotifyOAuth
import spotipy
import spotipy.util as util

from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import linear_kernel

In [17]:
top20_csv = pd.read_csv('../data/top20.csv', index_col=0)
candidatos_csv = pd.read_csv('../data/candidatos.csv', index_col=0)

In [18]:
top20_mtx = top20_csv.iloc[:,1:].values
candidatos_mtx = candidatos_csv.iloc[:,1:].values

In [19]:
scaler = StandardScaler()
t20_scaled = scaler.fit_transform(top20_mtx)
can_scaled = scaler.fit_transform(candidatos_mtx)

In [20]:
t20_norm = np.sqrt((t20_scaled*t20_scaled).sum(axis=1))
can_norm = np.sqrt((can_scaled*can_scaled).sum(axis=1))

In [21]:
nt20 = t20_scaled.shape[0]
ncan = can_scaled.shape[0]
t20 = t20_scaled/t20_norm.reshape(nt20,1)
can = can_scaled/can_norm.reshape(ncan,1)

In [22]:
cos_sim = linear_kernel(t20,can)
cos_sim.shape

(20, 598)

In [23]:
print(cos_sim[6,470])
print(cos_sim[3,54])

0.08765414690859012
0.45383298868460886


In [24]:
def obtener_candidatos(pos, cos_sim, ncands, umbral = 0.8):
    idx = np.where(cos_sim[pos,:]>=umbral)[0] 
    
    idx = idx[np.argsort(cos_sim[pos,idx])[::-1]]
    if len(idx) >= ncands:
        cands = idx[0:ncands]
    else:
        cands = idx
  
    return cands

In [25]:
for i in range(5):
    cands = obtener_candidatos(i, cos_sim, 5)
    print(f'{i}==> pistas filtradas: {cands}, similitudes: {cos_sim[i,cands]}')

0==> pistas filtradas: [156 500 431 391 501], similitudes: [0.90893065 0.90745715 0.89998005 0.89142521 0.88512955]
1==> pistas filtradas: [450 449 177], similitudes: [0.88125095 0.81584869 0.80191905]
2==> pistas filtradas: [358 193], similitudes: [0.81164574 0.81073033]
3==> pistas filtradas: [193], similitudes: [0.95609705]
4==> pistas filtradas: [], similitudes: []


In [26]:
ids_t20 = []
ids_playlist = []

for i in range(top20_csv.shape[0]):
    print(top20_csv.index[i])   
    ids_t20.append(top20_csv['id'][i])
    
    cands = obtener_candidatos(i, cos_sim, 5, umbral=0.8)
    

    if len(cands)==0:
        print('     ***No se encontraron pistas relacionadas***')
    else:
        for j in cands:
            id_cand = candidatos_csv['id'][j]
            ids_playlist.append(id_cand)
            
            print(f'   {candidatos_csv.index[j]}')

Si No Estás
   Like A Record
   Doesn’t Feel Like Christmas
   Hermosa Flor - Live
   Loving Means Leaving
   What Are You Doing New Year’s Eve?
QLONA
   Party Encendido
   EN 4
   Uwolnić Bonusa
greedy
   COCO LOCO
   Intro (Venezuela)
Intro (Venezuela)
   Intro (Venezuela)
Fire On Fire
     ***No se encontraron pistas relacionadas***
MI EX TENÍA RAZÓN
   Junto al Amanecer - Club Version
Sin Tiempo Para Bailar
   Sin Tiempo Para Bailar
   Running Away
Tamagotchi
   GUAYETEO
LALA
   hawaii
   Nari (feat. VillaBanks)
In The Stars
   Movies
   Mil y una noches
georgia
   Still
   Amanita
   Mil y una noches
Running Away
   Running Away
Read your Mind
   Read your Mind
   Flowers
   Wild Heart
   Something Unreal
   Santa's Coming for Us
Loving Means Leaving
   Loving Means Leaving
   Sad Forever
   la puerta pequeña
   Your Kiss
   Someone u don't know
Peligrosa
   Peligrosa
   Apretaito
   Charlie Be Quiet!
   Magnum
   Looks Like Me
Magnum
   Magnum
   La Tóxica
   SINCERAMENTE
   Ever

C:\Users\Felix\AppData\Local\Temp\ipykernel_20208\3808811813.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ids_t20.append(top20_csv['id'][i])
C:\Users\Felix\AppData\Local\Temp\ipykernel_20208\3808811813.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  id_cand = candidatos_csv['id'][j]


In [27]:
ids_playlist_dep = [x for x in ids_playlist if x not in ids_t20]
ids_playlist_dep = list(set(ids_playlist_dep))

In [28]:
cid = os.getenv('SPOTIPY_CLIENT_ID')
secret = os.getenv('SPOTIPY_CLIENT_SECRET')
redirect_uri = 'http://localhost:8080'
username = os.getenv('USERNAME')

scope = 'user-top-read playlist-modify-public playlist-modify-private'
auth_manager = SpotifyOAuth(client_id=cid, client_secret=secret, redirect_uri=redirect_uri, scope=scope, username=username)
sp = spotipy.Spotify(auth_manager=auth_manager)

pl = sp.user_playlist_create(user = username, 
                            name = 'Spotipy Recommender Playlist',
                            description = 'Playlist created with the recommendation system')

sp.playlist_add_items(pl['id'],ids_playlist_dep)

In [29]:
sp.playlist_add_items(pl['id'],ids_playlist_dep)

{'snapshot_id': 'MyxlY2JjMTZkYTc3NzEyYWQwMGU3Y2IyMDBkNTJiYWFkYjdlYjRkZjI1'}